In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn import tree
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score, precision_score
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingClassifier

#StackingClassifier

%matplotlib inline

In [69]:
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Новосиб/train_dataset_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Новосиб/test_dataset_test.csv")
df_solution = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Новосиб/sample_solution.csv")

In [71]:
df_train['Class'] = df_train['Class'].map({0:0, 1:1, 3:2, 4:3, 5:4, 64:5})
df_solution['Class'] = df_solution['Class'].map({0:0, 1:1, 3:2, 4:3, 5:4, 64:5})

In [72]:
df_train = df_train.fillna(0)

In [73]:
X = df_train.drop(["Class", "id", "Reflectance"], axis = 1)
y = df_train[["Class"]]

In [74]:
X_test = df_test.drop(["Reflectance", "id"], axis = 1)
test_id = df_test["id"]

In [75]:
ss = StandardScaler()
X_scaled = ss.fit_transform(X)
X_t_scaled = ss.fit_transform(X_test)

In [78]:
df_train['neighr'] = 0
neigh = RadiusNeighborsClassifier(radius=0.034)
neigh.fit(X_scaled, y)
for i in range(1, len(X_scaled) // 200000 + 2):
  print(i)
  df_train.iloc[(i - 1) * 200000:i * 200000, -1] = list(map(len, neigh.radius_neighbors(X_scaled[(i - 1) * 200000:i * 200000], 0.034, return_distance=False)))

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:501: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._fit(X, y)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [79]:
df_test['neighr'] = 0
for i in range(1, len(X_t_scaled) // 200000 + 2):
  print(i)
  df_test.iloc[(i - 1) * 200000:i * 200000, -1] = list(map(len, neigh.radius_neighbors(X_t_scaled[(i - 1) * 200000:i * 200000], 0.034, return_distance=False)))

1
2
3
4
5
6
7
8
9
10


In [80]:
X = df_train.drop(["Class", "id"], axis = 1)
y = df_train[["Class"]]

In [81]:
X_test = df_test.drop(["id"], axis = 1)
test_id = df_test["id"]

In [58]:
params = { 'max_depth': [10],
           'learning_rate': [0.01, 0.05, 0.1],
           'colsample_bytree': [0.3, 0.7]}

In [89]:
df_sub = pd.merge(df_solution, df_test, on="id", how="left")
df_sub = df_sub.drop(["Class", "id"], axis = 1)

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7575)

## Обучение модели

In [ ]:
clfXGBoost = XGBClassifier(tree_method="gpu_hist")
clf = GridSearchCV(estimator=clfXGBoost, 
                   param_grid=params,
                   scoring='recall_macro', 
                   verbose=1,
                   return_train_score=True,
                   cv=2)
clf.fit(X, y)
print("Best parameters:", clf.best_params_)

In [83]:
clfXGBoost = XGBClassifier(tree_method="gpu_hist", max_depth = 10)

In [86]:
estimators = [
     ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
     ('xgb', XGBClassifier(tree_method="gpu_hist", max_depth = 10))]
clf = StackingClassifier(estimators, final_estimator=RandomForestClassifier(n_estimators=10, random_state=7575), cv = 3)

In [87]:
clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


StackingClassifier(cv=3,
                   estimators=[('rf',
                                RandomForestClassifier(n_estimators=10,
                                                       random_state=42)),
                               ('xgb',
                                XGBClassifier(max_depth=10,
                                              tree_method='gpu_hist'))],
                   final_estimator=RandomForestClassifier(n_estimators=10,
                                                          random_state=7575))

In [90]:
pred = clf.predict(df_sub)

In [91]:
_id = df_solution["id"]
sub = pd.DataFrame({'id': _id , 'Class': pred})
sub['Class'] = sub['Class'].map({0.0:0, 1.0:1, 2.0:3, 3.0:4, 4.0:5, 5.0:64})
sub.to_csv('sub.csv', index=False)